## Imports

In [197]:
import pandas as pd
import numpy as np
import scipy.stats
import seaborn as sns
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import matplotlib.pyplot as plt

## Q1

1. It is not reasonable since the portfolio is constructed not just based on the Sharpe Ratio. The portfolio takes the covariance into account and optimized itself by minimizing the volatility (covariance) to fulfill a target return. Just comparing the Sharpe Ratios does not make sense to include crypto in our profile. 

2. False. Mean returns have a linear relationship, so I would not say that a tiny change in the mean will affect a lot the mean variance frontier. 

3. The ones using normal approxiamations are better in real data. From HW3, our group showed that the normal methods give a result that is closer to the actual quantile (0.05). In addition, historical simulation requires a large data sample to estimate. If the sample is small, then there will be large standard errors on the order statistic. 

4. 

5. Classic MV Portfolio only provide the customer two options. That is if you want to have higher return with higher volatility, you may need to invest more in tangency portfolio. If you don't want risk, then invest more in the GMV. However, it does not take the betas into account like Ridge and LASSO. Therefore, we might get more information from the Ridge about the beta estimates. 

6. 
   HFRIFWI - Hedge fund Fund Weighted Composite Index. Index of surveyed hedge funds (2000 Holdings) designed to track the performance of the hedge market as a whole. Monthly   returns based on surveys. This can't be launched as an ETF as they need a third party index as benchmark. 
   
   HDG - Actual tradaeble asset trying to track the MLFM-ES, by actually trading on these assets. HDG do not strictly adhere to MLFM-ES replication and would deviate from positions if they can get additional returns or reduce transaction cost. 



## Helper Functions

In [198]:
def performance_summary(return_data, annualization = 12):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*annualization)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(annualization))
    summary_stats['Sharpe Ratio/Mean over Volatility'] = summary_stats['Mean']/summary_stats['Volatility']

    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

In [199]:
def mvo_performance_stats(asset_returns,cov_matrix,port_weights, port_type,period):
    """ 
        Returns the Annualized Performance Stats for given asset returns, portfolio weights and covariance matrix
        Inputs: 
            asset_return - Excess return over the risk free rate for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
            port_weights = weights of the assets in the portfolio (1 x n) Vector
            port_type = Type of Portfolio | Eg - Tangency or Mean-Variance Portfolio
            period = Monthly frequency
    """
    
    ret = np.dot(port_weights,asset_returns)*period
    vol = np.sqrt(port_weights @ cov_matrix @ port_weights.T)*np.sqrt(period)
    sharpe = ret/vol

    stats = pd.DataFrame([[ret,vol,sharpe]],columns= ["Annualized Return","Annualized Volatility","Annualized Sharpe Ratio"], index = [port_type])
    return stats

In [200]:
def tangency_portfolio_rfr(asset_return,cov_matrix, cov_diagnolize = False):
    """ 
        Returns the tangency portfolio weights in a (1 x n) vector
        Inputs: 
            asset_return - return for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
    """
    if cov_diagnolize:
        asset_cov = np.diag(np.diag(cov_matrix))
    else:
        asset_cov = np.array(cov_matrix)
    inverted_cov= np.linalg.inv(asset_cov)
    one_vector = np.ones(len(cov_matrix.index))
    
    den = (one_vector @ inverted_cov) @ (asset_return)
    num =  inverted_cov @ asset_return
    return (1/den) * num

In [201]:
def mv_portfolio_rfr(asset_return,cov_matrix,target_ret,tangency_port):
    """ 
        Returns the Mean-Variance portfolio weights in a (1 x n) vector when a riskless assset is available
        Inputs: 
            asset_return - Excess return over the risk free rate for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
            target_ret = Target Return (Annualized)
            tangency_port = Tangency portfolio when a riskless assset is available
    """
    asset_cov = np.array(cov_matrix)
    inverted_cov= np.linalg.inv(asset_cov)
    one_vector = np.ones(len(cov_matrix.index))
    
    delta_den = (asset_return.T @ inverted_cov) @ (asset_return)
    delta_num = (one_vector @ inverted_cov) @ (asset_return)
    delta_tilde = (delta_num/delta_den) * target_ret
    return (delta_tilde * tangency_port)

In [202]:
def gmv_portfolio(asset_return,cov_matrix):
    """ 
        Returns the Global Minimum Variance portfolio weights in a (1 x n) vector
        Inputs: 
            asset_return - return for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
    """
    asset_cov = np.array(cov_matrix)
    inverted_cov= np.linalg.inv(asset_cov)
    one_vector = np.ones(len(cov_matrix.index))
    
    den = (one_vector @ inverted_cov) @ (one_vector)
    num =  inverted_cov @ one_vector
    return (1/den) * num

In [203]:
def mv_portfolio(asset_return,cov_matrix,target_ret,tangency_port):
    """ 
        Returns the Mean-Variance portfolio weights in a (1 x n) vector when NOOOOO riskless assset is available
        Inputs: 
            asset_return - total return for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
            target_ret = Target Return (Not-Annualized)
            tangency_port = Tangency portfolio
    """
    omega_tan = tangency_portfolio_rfr(asset_return.mean(),cov_matrix)
    omega_gmv = gmv_portfolio(asset_return,cov_matrix) 
    
    mu_tan = asset_return.mean() @ omega_tan
    mu_gmv = asset_return.mean() @ omega_gmv
    
    delta = (target_ret - mu_gmv)/(mu_tan - mu_gmv)
    mv_weights = delta * omega_tan + (1-delta)*omega_gmv
    return mv_weights

In [204]:
def regression_based_performance(factor,fund_ret,rf,constant = True):
    """ 
        Returns the Regression based performance Stats for given set of returns and factors
        Inputs:
            factor - Dataframe containing monthly returns of the regressors
            fund_ret - Dataframe containing monthly excess returns of the regressand fund
            rf - Monthly risk free rate of return
        Output:
            summary_stats - (Beta of regression, treynor ratio, information ratio, alpha). 
    """
    if constant:
        X = sm.tools.add_constant(factor)
    else:
        X = factor
    y=fund_ret
    model = sm.OLS(y,X,missing='drop').fit()
    
    if constant:
        beta = model.params[1:]
        alpha = round(float(model.params['const']),6)
        
    else:
        beta = model.params
    treynor_ratio = ((fund_ret.values-rf.values).mean()*12)/beta[0]
    tracking_error = (model.resid.std()*np.sqrt(12))
    if constant:        
        information_ratio = model.params[0]*12/tracking_error
    r_squared = model.rsquared
    if constant:
        return (beta,treynor_ratio,information_ratio,alpha,r_squared,tracking_error)
    else:
        return (beta,treynor_ratio,r_squared,tracking_error)

In [205]:
def rolling_regression_param(factor,fund_ret,roll_window = 60):
    """ 
        Returns the Rolling Regression parameters for given set of returns and factors
        Inputs:
            factor - Dataframe containing monthly returns of the regressors
            fund_ret - Dataframe containing monthly excess returns of the regressand fund
            roll_window = rolling window for regression
        Output:
            params - Dataframe with time-t as the index and constant and Betas as columns
    """
    X = sm.add_constant(factor)
    y= fund_ret
    rols = RollingOLS(y, X, window=roll_window)
    rres = rols.fit()
    params = rres.params.copy()
    params.index = np.arange(1, params.shape[0] + 1)
    return params

## Import Data

In [206]:
total_ret = pd.read_excel('midterm_1.xlsx',sheet_name = 'total returns', index_col = 'date')
total_ret.head()

,CL1,GC1,KC1,ES1,BP1
date,,,,,
2009-01-31,-0.113532,0.048627,0.061111,-0.086109,-0.007831
2009-02-28,0.044680,0.015187,-0.079479,-0.107294,-0.008309
2009-03-31,0.087892,-0.021111,0.034402,0.087209,0.001745
2009-04-30,-0.013826,-0.036543,-0.006491,0.094679,0.032897
2009-05-31,0.287437,0.098300,0.185518,0.055172,0.088934


## Q2

In [207]:
total_mean = total_ret.mean() 
total_cov = total_ret.cov() 
gmv_port = gmv_portfolio(total_mean, total_cov)
tan_port = tangency_portfolio_rfr(total_mean, total_cov) 
gmv_port_df = pd.DataFrame(tan_port,columns= ["GMV Portfolio Weight"],index=total_ret.columns) 
tan_port_df = pd.DataFrame(tan_port,columns= ["Tangency Portfolio Weight"],index=total_ret.columns) 


In [208]:
gmv_port_df

,GMV Portfolio Weight
CL1,-0.128124
GC1,1.191087
KC1,0.097813
ES1,4.220019
BP1,-4.380796


In [209]:
tan_port_df

,Tangency Portfolio Weight
CL1,-0.128124
GC1,1.191087
KC1,0.097813
ES1,4.220019
BP1,-4.380796


In [210]:
mv_port = mv_portfolio(total_ret, total_cov, 0.02/12, tan_port) 
mv_port_df = pd.DataFrame(mv_port,columns= ["Mean-Variance Portfolio Weight"],index=total_ret.columns) 
mv_port_df

,Mean-Variance Portfolio Weight
CL1,-0.031961
GC1,0.192479
KC1,-0.009207
ES1,0.146680
BP1,0.702010


In [211]:
performance = mvo_performance_stats(total_mean, total_cov, mv_port, 'Mean-Variance', 12) 
performance

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio
Mean-Variance,0.02,0.079349,0.252052


From the performance table, you may find that the annualized volatility is 7.93%. 

In [212]:
mv_port_rf = mv_portfolio_rfr(total_mean, total_cov, 0.02/12, tan_port) 
mv_port_rf_df = pd.DataFrame(mv_port_rf,columns= ["Risk Free Mean-Variance Portfolio Weight"],index=total_ret.columns) 
performance_rf = mvo_performance_stats(total_mean, total_cov, mv_port_rf, 'Mean-Variance', 12) 
performance_rf

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio
Mean-Variance,0.02,0.018236,1.096747


The solution has less volatility since investing in risk free helps us reduce it. This makes sense as the risk free asset has no risk and it represents a portion in the portfolio. 

## Q3

In [213]:
y = total_ret['ES1']
x = sm.add_constant(total_ret['BP1'])

hedge_reg = RollingOLS(y,x,window=36).fit() 
rolling_betas = hedge_reg.params.copy()
table = hedge_reg.params
table.tail(5)
    

,const,BP1
date,,
2022-02-28,0.013660,1.178892
2022-03-31,0.014433,1.144168
2022-04-30,0.012270,1.247557
2022-05-31,0.013008,1.186588
2022-06-30,0.010191,1.251756


In [214]:
r = (rolling_betas * x).sum(axis=1,skipna=False)
table['r*'] = r
table.tail(5)

,const,BP1,r*
date,,,
2022-02-28,0.013660,1.178892,0.010155
2022-03-31,0.014433,1.144168,-0.009789
2022-04-30,0.012270,1.247557,-0.038274
2022-05-31,0.013008,1.186588,0.013573
2022-06-30,0.010191,1.251756,-0.030127


In [215]:
table1 = table.dropna() 
r1 = table1.drop(columns=['const','BP1'])
summary_stats = r1.mean().to_frame('Mean').apply(lambda x: x*12)
summary_stats['Volatility'] = r1.std().apply(lambda x: x*np.sqrt(12))
summary_stats['Mean over Volatility'] = summary_stats['Mean']/summary_stats['Volatility'] 
summary_stats

,Mean,Volatility,Mean over Volatility
r*,0.126333,0.062546,2.019829


In [216]:
r1['ES1'] = total_ret['ES1'] 
r1.corr()

,r*,ES1
r*,1.000000,0.521038
ES1,0.521038,1.000000


ES1 and r* are positively correlated. 

## Q4

In [217]:
total_ret['50/50'] = 0.5 * total_ret['ES1'] + 0.5 * total_ret['GC1'] 
mr = total_ret.drop(columns=['CL1', 'KC1', 'BP1'])
mr.head()

,GC1,ES1,50/50
date,,,
2009-01-31,0.048627,-0.086109,-0.018741
2009-02-28,0.015187,-0.107294,-0.046053
2009-03-31,-0.021111,0.087209,0.033049
2009-04-30,-0.036543,0.094679,0.029068
2009-05-31,0.098300,0.055172,0.076736


In [218]:
performance_mr = performance_summary(mr)
performance_mr

,Mean,Volatility,Sharpe Ratio/Mean over Volatility,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Max Drawdown,Peak,Bottom,Recovery
GC1,0.054008,0.160134,0.337267,0.113357,0.142579,-0.065664,-0.085920,-0.429597,2011-08-31,2015-12-31,NaT
ES1,0.129117,0.151067,0.854698,-0.439491,0.723828,-0.071244,-0.091453,-0.203174,2019-12-31,2020-03-31,2020-07-31
50/50,0.091562,0.114079,0.802619,0.109368,0.253804,-0.045782,-0.058951,-0.120982,2022-03-31,2022-06-30,NaT


From the table, ES1 has the highest Mean, 50/50 has the lowest volatility, ES1 has the highest mean/vol, but 50/50 has the best Max Drawdown. 

This is not what we want from the portfolio theory as the ES1 has the best performance by itself except for the Max Drawdown. Simple mixing of two portfolios does not give better performance. It only reduces the chance of the portfolio to behave really bad. 

In [219]:
normal_1_z = scipy.stats.norm().ppf(0.01)
normal_1_pdf = scipy.stats.norm().pdf(normal_1_z)
data = mr['50/50']

In [220]:
cvar_full = -normal_1_pdf/0.01*data.std()
cvar_full

-0.08777058366877924

In [221]:
std = data.rolling(24,min_periods=24).std().shift(1).to_frame()
cvar_rolling = (-normal_1_pdf/0.01*std)
cvar_rolling.dropna().tail()

,50/50
date,
2022-02-28,-0.108114
2022-03-31,-0.102707
2022-04-30,-0.094743
2022-05-31,-0.091618
2022-06-30,-0.091609


To get that ES1 out perform 50/50, we are actually calculate P(R[ES1] > R[50/50]).

In [222]:
def find_prob(mean, vol, periods):
    prob = pd.Series(scipy.stats.norm.cdf(-np.sqrt(periods)*mean/vol))
    prob.index = periods
    return prob

In [223]:
log_mr = np.log(1+mr)

mr_ex = log_mr['ES1'] - log_mr['50/50'] 
mean = mr_ex.mean()*12 
vol = mr_ex.std()*np.sqrt(12)
mean

0.03229975374488571

In [224]:
prob_data = find_prob(mean, vol, np.array([10]))
prob_data

10    0.168365
dtype: float64

This shows that we have 17% of the chance that ES1 will underperform 50/50 which is reasonable, since we expect that ES1 itself will perform better in reality. 